In [1]:
%load_ext autoreload

In [2]:
from pathlib import Path
import copy
from datetime import datetime, timedelta
import os

from darpinstances.instance_generation.generate_config import generate_config, default_config

In [3]:
def generate_instance(area: dict, start_time_str: str, duration: int, max_delay: int):
    config = copy.deepcopy(shared_config)
    config['max_prolongation'] = max_delay * 60
    config['map']['SRID_plane'] = area['srid_plane']
    config['demand']['min_time'] = f"{experiment_date} {start_time_str}:00"
    config['demand']['dataset'] = area['demand datasets']
    config['demand']['positions_set'] = area['position_set']
    config['area_id'] = area['id']
    if 'trip_time_set' in area:
        config['demand']['time_set'] = area['trip_time_set']

    start_time = datetime.strptime(config['demand']['min_time'], '%Y-%m-%d %H:%M:%S')

    vehicle_start_time = start_time - timedelta(minutes=area['vehicle_start_interval'])
    config['vehicles']['start_time'] = vehicle_start_time.strftime('%Y-%m-%d %H:%M:%S')

    end_time = start_time + timedelta(minutes=duration)
    config['demand']['max_time'] = end_time.strftime('%Y-%m-%d %H:%M:%S')

    if duration >= 60:
        duration_folder_name = f"duration_{int(duration / 60):0>2d}_h"
    elif duration >= 1:
        duration_folder_name = f"duration_{duration:0>2d}_min"
    else:
        duration_folder_name = f"duration_{int(duration * 60):0>2d}_s"

    instance_dir = root_path / Path(area['name']) / Path('instances') / Path(f"start_{start_time.hour:0>2d}-00") \
                   / Path(duration_folder_name) / Path(f"max_delay_{max_delay:0>2d}_min")
    os.makedirs(instance_dir, exist_ok=True)
    config_path = instance_dir / Path("config.yaml")

    generate_config(config, config_path)



# Configuration

In [4]:
root_path = Path(r'C:\Google Drive\AIC Experiment Data\DARP\ITSC_instance_paper/Instances/')
# root_path = Path(r'D:\Google Drive\AIC Experiment Data\DARP\ITSC_instance_paper/Instances/')

experiment_date = '2022-04-05' # Tuesday, the most congested day of the most congested month in 2022 in Washington DC

max_delays = [3, 5, 10]

# cities
# - Washington DC: ~8 000 requests per day
# - New York: ~650 000 requests per day
# - Chicago: ~15 000 requests per day
areas = [
    {'name': 'DC', 'demand datasets': [7], 'id': 22, 'zone_types': [4], 'srid_plane': 32618, 'position_set': 4, 'vehicle_start_interval': 11, 'trip_time_set': 2},
    {'name': 'NYC', 'demand datasets': [2, 3, 4, 5], 'id': 12, 'zone_types': [2], 'srid_plane': 32618, 'position_set': 1, 'vehicle_start_interval': 24},
    {'name': 'Manhattan', 'demand datasets': [2, 3, 4, 5], 'id': 4, 'zone_types': [2], 'srid_plane': 32618, 'position_set': 1, 'vehicle_start_interval': 8},
    {'name': 'Chicago', 'demand datasets': [1], 'id': 19, 'zone_types': [0, 1], 'srid_plane': 26916, 'position_set': 3, 'vehicle_start_interval': 31, 'trip_time_set': 1},
]

shared_config = copy.deepcopy(default_config)
shared_config['demand']['mode'] = 'load'
shared_config['vehicles']['vehicle_to_request_ratio'] = 1
shared_config['area_dir'] = "../../../../"
shared_config['demand']['filepath'] = "./trips.di"

## Start 18:00 -  Short Instances (up to two hours)

In [13]:
durations = [0.5, 1, 2, 5, 15, 30, 120] # in minutes, up to 16 hours
start_times = ['18:00']

## Start 07:00 - Long Instances (16 hours)

In [5]:
durations = [960] # in minutes, up to 16 hours
start_times = ['07:00']

# Running the instance generation

In [6]:
for area in areas:
    for start_time in start_times:
        for duration in durations:
            for max_delay in max_delays:
                generate_instance(area, start_time, duration, max_delay)

10:20:19 [INFO] Saving config to C:\Google Drive\AIC Experiment Data\DARP\ITSC_instance_paper\Instances\DC\instances\start_07-00\duration_16_h\max_delay_03_min\config.yaml
10:20:19 [INFO] Saving config to C:\Google Drive\AIC Experiment Data\DARP\ITSC_instance_paper\Instances\DC\instances\start_07-00\duration_16_h\max_delay_05_min\config.yaml
10:20:19 [INFO] Saving config to C:\Google Drive\AIC Experiment Data\DARP\ITSC_instance_paper\Instances\DC\instances\start_07-00\duration_16_h\max_delay_10_min\config.yaml
10:20:19 [INFO] Saving config to C:\Google Drive\AIC Experiment Data\DARP\ITSC_instance_paper\Instances\NYC\instances\start_07-00\duration_16_h\max_delay_03_min\config.yaml
10:20:19 [INFO] Saving config to C:\Google Drive\AIC Experiment Data\DARP\ITSC_instance_paper\Instances\NYC\instances\start_07-00\duration_16_h\max_delay_05_min\config.yaml
10:20:19 [INFO] Saving config to C:\Google Drive\AIC Experiment Data\DARP\ITSC_instance_paper\Instances\NYC\instances\start_07-00\duration